# Лінійна регресія на California Housing Dataset
**Автор:** Moiseyenko Svitlana  
**Мета:** Побудова моделі лінійної регресії з покращеними метриками через очистку викидів та нормалізацію ознак

## 1. Імпорт необхідних пакетів

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from scipy.stats import zscore

## 2. Завантаження набору даних California Housing

In [ ]:
california = fetch_california_housing(as_frame=True)
data = california.frame

print(f'Розмір датасету: {data.shape}')
print(f'\nПерші 5 рядків:')
data.head()

## 3.1. Очистка від викидів за методом zscore

In [ ]:
outlier_cols = ['AveRooms', 'AveBedrms', 'AveOccup', 'Population']
z_scores = data[outlier_cols].apply(zscore)
anomalies = (np.abs(z_scores) > 3).any(axis=1)

data_clean = data[~anomalies].copy()

removed_count = anomalies.sum()
print(f'Видалено рядків з аномальними значеннями: {removed_count}')
print(f'Розмір датасету після очистки: {data_clean.shape}')
print(f'Залишилось: {(1 - removed_count / len(data)) * 100:.1f}% даних')

## 3.2. Видалення ознаки з високою кореляцією

In [ ]:
data_clean = data_clean.drop(columns=['AveBedrms'])

print('Видалено ознаку: AveBedrms (висока кореляція з AveRooms)')
print(f'Кількість ознак після видалення: {data_clean.shape[1] - 1}')

## 4. Розподіл на навчальну та тестову вибірки

In [ ]:
X = data_clean.drop(columns=['MedHouseVal'])
y = data_clean['MedHouseVal']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Навчальна вибірка: {X_train.shape[0]} зразків')
print(f'Тестова вибірка: {X_test.shape[0]} зразків')
print(f'Кількість ознак: {X_train.shape[1]}')

## 5. Нормалізація ознак

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('Нормалізація завершена')
print(f'Середнє значення ознак (навчальна вибірка): {X_train_scaled.mean():.2e}')
print(f'Стандартне відхилення ознак (навчальна вибірка): {X_train_scaled.std():.2f}')

## 6. Побудова моделі лінійної регресії

In [ ]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

print('Модель навчена успішно')
print(f'Кількість коефіцієнтів: {len(model.coef_)}')

## 7. Оцінка показників моделі

In [ ]:
r_sq = model.score(X_train_scaled, y_train)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print('=== Метрики моделі ===')
print(f'R2: {r_sq:.2f} | MAE: {mae:.2f} | MAPE: {mape:.2f}')

## 8. Порівняння з базовою моделлю та висновки

In [ ]:
baseline_r2 = 0.60
baseline_mae = 0.53
baseline_mape = 0.32

print('=== Порівняння метрик ===')
print(f'\nR² (коефіцієнт детермінації):')
print(f'  Базова модель: {baseline_r2:.2f}')
print(f'  Покращена модель: {r_sq:.2f}')
print(f'  Покращення: {(r_sq - baseline_r2) * 100:.1f} п.п.')

print(f'\nMAE (середня абсолютна похибка):')
print(f'  Базова модель: {baseline_mae:.2f}')
print(f'  Покращена модель: {mae:.2f}')
print(f'  Зменшення похибки: {(baseline_mae - mae) / baseline_mae * 100:.1f}%')

print(f'\nMAPE (середня абсолютна похибка у %):')
print(f'  Базова модель: {baseline_mape:.2f}')
print(f'  Покращена модель: {mape:.2f}')
print(f'  Зменшення похибки: {(baseline_mape - mape) / baseline_mape * 100:.1f}%')

print('\n=== Висновки ===')
print('1. Очистка викидів дозволила зменшити вплив аномальних значень на модель')
print('2. Видалення корельованої ознаки AveBedrms зменшило мультиколінеарність')
print('3. Нормалізація ознак покращила збіжність алгоритму навчання')
print(f'4. Загальне покращення метрик становить ~{(r_sq - baseline_r2) * 100:.0f}-{(baseline_mape - mape) / baseline_mape * 100:.0f}%')